In [1]:
# Import packages
%matplotlib inline
import pandas as pd
import numpy as np
import geopandas
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.tree import export_graphviz
from IPython.display import SVG
from graphviz import Source
from IPython.display import display

In [2]:
# Import and Organize Tree Survey

# Import tree survey data
data = geopandas.read_file('Data/Trees/2015 Street Tree Census - Tree Data/geo_export_f9cac9c3-66c4-45b6-899b-823a270d8754.shp')

# Turn tree survey data into a normal dataframe and remove a bunch of columns
data_nogeom = pd.DataFrame(data.drop(columns='geometry'), copy=True)
data_nogeom=data_nogeom.drop(['block_id', 'boro_ct', 'borocode', 'boroname', 'brnch_ligh',
       'brnch_othe', 'brnch_shoe', 'cb_num', 'cncldist','nta',
       'nta_name','spc_latin', 'st_assem', 'st_senate', 'state','stump_diam','user_type', 'x_sp', 'y_sp', 'zip_city'],axis=1)

data_nogeom=data_nogeom.dropna()

In [3]:
# Import and organize 311 tree fall data and merge with tree survey data

# Import fall data and drop irrelevant columns
data_falls = pd.read_csv('Data/311/Tree_Fall_311.csv',quoting=3)
data_falls = data_falls.drop(['Incident Zip',
       'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Status', 'Borough',
       'X Coordinate (State Plane)', 'Y Coordinate (State Plane)'],axis=1)

# Remove the falls before 2015 (before survey)
data_falls['Created Date']= pd.to_datetime(data_falls['Created Date'])
data_falls = data_falls.where(data_falls['Created Date']>'2015-01-01')

# Merge falls and tree survey based on address
data_with_falls = data_falls.merge(data_nogeom,how="right",left_on="Incident Address",right_on="address")

# Create column Fall to have True/False for if tree branch or full tree fell
data_with_falls['Fall']=  list(map(lambda x: not np.isnan(x),data_with_falls['Unique Key']))
data_with_falls['Fall'].head()


C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3057: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0    True
1    True
2    True
3    True
4    True
Name: Fall, dtype: bool

In [4]:
# Import and organize 311 tree damage warning data and merge with tree survey data

# Import reported damage warning data and drop unneeded columns
data_damage = pd.read_csv('Data/311/Tree_Damage_311.csv')
data_damage = data_damage.drop(['Incident Zip',
       'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Status', 'Borough',
       'X Coordinate (State Plane)', 'Y Coordinate (State Plane)'],axis=1)

# Merge in tree warning data based on address
data_with_falls_damage = data_damage.merge(data_with_falls,how="right",left_on="Incident Address",right_on="address")

# Create column Damage to have True/False for if any damage was reported
data_with_falls_damage['Damage']=  list(map(lambda x: not np.isnan(x),data_with_falls_damage['Unique Key_x']))
